In [1]:
# 유튜브 랭킹(https://youtube-rank.com/board/bbs/board.php?bo_table=youtube) 사이트를 크롤링하여 다음에 답하세요.
# 1. 구독자순 기준으로 Top 1000 을 크롤링하여 
#    순위, 카테고리, 채널명, 구독자수, 조회수, 비디오수 필드를 데이터프레임으로 
#    만드세요.(20)

# 2. 웹 서버를 만들고 localhost:5000/youtube_ranking을 주소창에 입력하면 
#    유튜브 랭킹을 크롤링한 결과를 표로 보여주는 코드를 작성하고
#    결과 웹화면을 스캔해서 답안지에 포함하세요.(10)

# 3. 구독자수 Top 20 채널, 카테고리별 채널수 Top 10 카테고리를 
#    웹 서버를 이용하여 보여주는 코드를 작성하고 결과 웹화면을 스캔해서
#    답안지에 포함하세요.(20)

In [1]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [9]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + '1'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
res = requests.get(url,headers=header)
soup = BeautifulSoup(res.text,'html.parser')

In [4]:
len(res.text) 

292114

In [5]:
res.text

'<!doctype html>\n<html lang="ko">\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="imagetoolbar" content="no">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <title>유튜브 랭킹 1 페이지</title>\n    <link rel="stylesheet" href="/css/default.css?ver=2022022">\n<link rel="stylesheet" href="/css/bootstrap.css?ver=2022022">\n<link rel="stylesheet" href="/css/nifty.min.css?ver=2022022">\n<link rel="stylesheet" href="/css/premium/icon-sets/icons/line-icons/premium-line-icons.min.css?ver=2022022" >\n<link rel="stylesheet" href="/css/themes/type-c/theme-well-red.css?ver=2022022">\n<link rel="stylesheet" href="https://youtube-rank.com/board/theme/youtube/skin/outlogin/youtuberank/style.css?ver=2022022">\n<link rel="stylesheet" href="https://youtube-rank.com/board/skin/board/sns_channel/list.skin.css?2022022" />\n<link rel="stylesheet" href="https://unpkg.com/aos@2.3.1/dist/aos.css" />\n    <link rel="apple-touch-icon" sizes="180x180" href="/img/apple-touch-icon.png">\n    <link 

In [10]:
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe') # chromedriver 위치
driver.get(url)

C:\Users\user\AppData\Local\Temp\ipykernel_20748\2372343077.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe') # chromedriver 위치


In [7]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('.aos-init')
len(trs)

100

In [8]:
tr = trs[0]

In [9]:
# 순위, 카테고리, 채널명, 구독자수, 조회수, 비디오수
rank = tr.select_one('.rank').get_text().strip()
rank

'1'

In [10]:
# 분야
category = tr.select_one('.category').get_text().strip()[1:-1]
category

'음악/댄스/가수'

In [11]:
# 채널명
channel = tr.select_one('.subject > h1 > a').get_text().strip()
channel

'BLACKPINK'

In [12]:
#구독자순(정수로)
follow = tr.select_one('td.subscriber_cnt').get_text().strip()
follow = int(follow[:-1])*10000
follow

88000000

In [13]:
# View순(정수로)
view_str = tr.select_one('td.view_cnt').get_text().strip()
view_str

'309억5913만'

In [12]:
# 숫자로 바꿔주는 함수
def convert(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)

In [15]:
# Video순(정수로)
video = tr.select_one('td.video_cnt').get_text().strip()
video = int(video[:-1])
video

514

In [13]:
import time
lines = []
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(page)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    trs = soup.select('.aos-init')

    for tr in trs:
        rank =int(tr.select_one('.info_rank').get_text().strip())
        category=tr.select_one('.category').get_text().strip()[1:-1]
        title = tr.select_one('h1 > a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert(tr.select_one('.video_cnt').get_text().strip())
        lines.append({'순위':rank,'카테고리':category,'채널명':title,'구독자수':subscriber,
                    '조회수':view,'비디오':video})
driver.close()

In [14]:
df = pd.DataFrame(lines)

In [15]:
df.head()

,순위,카테고리,채널명,구독자수,조회수,비디오
0,1,음악/댄스/가수,BLACKPINK,88100000,30991890000,515
1,2,음악/댄스/가수,BANGTANTV,74700000,20287850000,2195
2,3,음악/댄스/가수,HYBE LABELS,70600000,27702240000,1186
3,4,음악/댄스/가수,SMTOWN,31700000,26937410000,4122
4,5,음악/댄스/가수,JYP Entertainment,26900000,19400680000,1679


In [17]:
df1 = df[:20]

In [18]:
df1

,순위,카테고리,채널명,구독자수,조회수,비디오
0,1,음악/댄스/가수,BLACKPINK,88100000,30991890000,515
1,2,음악/댄스/가수,BANGTANTV,74700000,20287850000,2195
2,3,음악/댄스/가수,HYBE LABELS,70600000,27702240000,1186
3,4,음악/댄스/가수,SMTOWN,31700000,26937410000,4122
4,5,음악/댄스/가수,JYP Entertainment,26900000,19400680000,1679
5,6,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],26500000,11052880000,223
6,7,음악/댄스/가수,1MILLION Dance Studio,26000000,7800900000,5390
7,8,음악/댄스/가수,1theK (원더케이),24500000,23708220000,18186
8,9,미분류,Alan Becker,22000000,5107370000,152
9,10,음악/댄스/가수,Mnet K-POP,20500000,14737540000,32243


In [21]:
# df1.to_csv(f'static/유튜브탑20.csv', index=False)

In [19]:
df

,순위,카테고리,채널명,구독자수,조회수,비디오
0,1,음악/댄스/가수,BLACKPINK,88100000,30991890000,515
1,2,음악/댄스/가수,BANGTANTV,74700000,20287850000,2195
2,3,음악/댄스/가수,HYBE LABELS,70600000,27702240000,1186
3,4,음악/댄스/가수,SMTOWN,31700000,26937410000,4122
4,5,음악/댄스/가수,JYP Entertainment,26900000,19400680000,1679
...,...,...,...,...,...,...
995,999,TV/방송,KBS여행 걸어서 세계속으로,730000,275970000,10964
996,998,미분류,빨간내복야코,730000,508340000,246
997,997,키즈/어린이,코코몽 COCOMONG TV - Cartoon & Song For Kids,730000,951630000,1420
998,996,취미/라이프,DanoTV,730000,158500000,642


In [27]:
df2 = df.pivot_table('채널명', '카테고리', aggfunc='count').sort_values('채널명', ascending=False)
df2 = df2[:10]
df2

,채널명
카테고리,
미분류,186
음악/댄스/가수,153
키즈/어린이,133
TV/방송,124
음식/요리/레시피,71
BJ/인물/연예인,59
뉴스/정치/사회,58
게임,51
취미/라이프,32


In [30]:
df2.to_csv(f'static/카테고리10.csv', index=True)

In [7]:
pd.read_csv("static/유튜브탑20.csv", sep=",")

,순위,카테고리,채널명,구독자수,조회수,비디오
0,1,음악/댄스/가수,BLACKPINK,88000000,30959130000,514
1,2,음악/댄스/가수,BANGTANTV,74600000,20279390000,2195
2,3,음악/댄스/가수,HYBE LABELS,70600000,27690310000,1185
3,4,음악/댄스/가수,SMTOWN,31700000,26928580000,4121
4,5,음악/댄스/가수,JYP Entertainment,26900000,19395230000,1678
5,6,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],26500000,11052880000,223
6,7,음악/댄스/가수,1MILLION Dance Studio,26000000,7799860000,5385
7,8,음악/댄스/가수,1theK (원더케이),24500000,23704760000,18177
8,9,미분류,Alan Becker,22000000,5103090000,152
9,10,음악/댄스/가수,Mnet K-POP,20500000,14733070000,32240


In [32]:
a = pd.read_csv("static/카테고리10.csv", sep=",")
a

,카테고리,채널명
0,미분류,186
1,음악/댄스/가수,153
2,키즈/어린이,133
3,TV/방송,124
4,음식/요리/레시피,71
5,BJ/인물/연예인,59
6,뉴스/정치/사회,58
7,게임,51
8,취미/라이프,32
9,영화/만화/애니,30


In [33]:
a.columns

Index(['카테고리', '채널명'], dtype='object')

In [37]:
a.iloc[0][0], a.iloc[0][1]

('미분류', 186)

In [39]:
a.iloc[1][0], a.iloc[1][1]

('음악/댄스/가수', 153)

In [40]:
a.iloc[2][0], a.iloc[2][1]

('키즈/어린이', 133)

In [45]:
dic=[]
for i in range(10):
    dic.append(a.iloc[i][0])
    dic.append(a.iloc[i][1])
dic


['미분류',
 186,
 '음악/댄스/가수',
 153,
 '키즈/어린이',
 133,
 'TV/방송',
 124,
 '음식/요리/레시피',
 71,
 'BJ/인물/연예인',
 59,
 '뉴스/정치/사회',
 58,
 '게임',
 51,
 '취미/라이프',
 32,
 '영화/만화/애니',
 30]

In [54]:
dic[::2]

['미분류',
 '음악/댄스/가수',
 '키즈/어린이',
 'TV/방송',
 '음식/요리/레시피',
 'BJ/인물/연예인',
 '뉴스/정치/사회',
 '게임',
 '취미/라이프',
 '영화/만화/애니']

In [55]:
dic[1::2]

[186, 153, 133, 124, 71, 59, 58, 51, 32, 30]

In [56]:
names = dic[::2]
values = dic[1::2]

In [57]:
my_dict = {names[i]: values[i] for i in range(len(names))}

In [58]:
my_dict

{'미분류': 186,
 '음악/댄스/가수': 153,
 '키즈/어린이': 133,
 'TV/방송': 124,
 '음식/요리/레시피': 71,
 'BJ/인물/연예인': 59,
 '뉴스/정치/사회': 58,
 '게임': 51,
 '취미/라이프': 32,
 '영화/만화/애니': 30}

In [59]:
my_dict.keys()

dict_keys(['미분류', '음악/댄스/가수', '키즈/어린이', 'TV/방송', '음식/요리/레시피', 'BJ/인물/연예인', '뉴스/정치/사회', '게임', '취미/라이프', '영화/만화/애니'])

In [60]:
for key in my_dict.keys():
    print(key)

미분류
음악/댄스/가수
키즈/어린이
TV/방송
음식/요리/레시피
BJ/인물/연예인
뉴스/정치/사회
게임
취미/라이프
영화/만화/애니


In [61]:
names

['미분류',
 '음악/댄스/가수',
 '키즈/어린이',
 'TV/방송',
 '음식/요리/레시피',
 'BJ/인물/연예인',
 '뉴스/정치/사회',
 '게임',
 '취미/라이프',
 '영화/만화/애니']

In [62]:
values

[186, 153, 133, 124, 71, 59, 58, 51, 32, 30]